<a href="https://colab.research.google.com/github/vinuprasath007/python.ipyb/blob/main/VINU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install keras-tuner

In [11]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [12]:
fashion_mnist = keras.datasets.fashion_mnist

In [13]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [14]:
train_images = train_images/255.0
test_images = test_images/255.0

In [15]:
train_images[0].shape

(28, 28)

In [16]:
test_images = test_images.reshape(len(test_images), 28, 28, 1)
train_images = train_images.reshape(len(train_images), 28, 28, 1)

In [17]:
def build_model(hp):
  model = keras.Sequential(
      [
       keras.layers.Conv2D(
           filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
           kernel_size = hp.Choice('conv_1_kernel', values= [3,5]),
           activation = 'relu',
           input_shape = (28, 28, 1)
       ),
       keras.layers.Conv2D(
           filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
           kernel_size = hp.Choice('conv_2_kernel', values= [3,5]),
           activation = 'relu',
       ),
       keras.layers.Flatten(),
       keras.layers.Dense(
           units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
           activation = 'relu'
       )
      ])
  model.compile( optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])), loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'])
  return model

In [18]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [19]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials = 1, directory = 'output', project_name = 'Mnist Fashion')

In [20]:
tuner_search.search(train_images, train_labels, epochs = 1, validation_split = 0.1)

Trial 1 Complete [00h 12m 12s]
val_accuracy: 0.15049999952316284

Best val_accuracy So Far: 0.15049999952316284
Total elapsed time: 00h 12m 12s
INFO:tensorflow:Oracle triggered exit


In [21]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        153648    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1115184   
Total params: 1,270,112
Trainable params: 1,270,112
Non-trainable params: 0
_________________________________________________________________
